In [1]:
%%time
import os
import threading
import subprocess
import numpy as np
from random import randrange
import math

CPU times: user 86.9 ms, sys: 4.17 ms, total: 91.1 ms
Wall time: 107 ms


In [2]:
source_dir = "../data/source/"
source_files = []
for j in sorted(os.listdir(source_dir)):
    source_files += [f"{source_dir}{j}"]
source_files

['../data/source/0_0.mkv',
 '../data/source/10_2700.mkv',
 '../data/source/11_3600.mkv',
 '../data/source/12_4500.mkv',
 '../data/source/13_5400.mkv',
 '../data/source/14_0.mkv',
 '../data/source/15_900.mkv',
 '../data/source/16_1800.mkv',
 '../data/source/17_2700.mkv',
 '../data/source/18_3600.mkv',
 '../data/source/19_4500.mkv',
 '../data/source/1_900.mkv',
 '../data/source/20_5400.mkv',
 '../data/source/21_6300.mkv',
 '../data/source/22_0.mkv',
 '../data/source/23_900.mkv',
 '../data/source/24_1800.mkv',
 '../data/source/25_2700.mkv',
 '../data/source/26_3600.mkv',
 '../data/source/27_4500.mkv',
 '../data/source/28_5400.mkv',
 '../data/source/29_6300.mkv',
 '../data/source/2_1800.mkv',
 '../data/source/30_7200.mkv',
 '../data/source/31_8100.mkv',
 '../data/source/32_9000.mkv',
 '../data/source/3_2700.mkv',
 '../data/source/4_3600.mkv',
 '../data/source/5_4500.mkv',
 '../data/source/6_5400.mkv',
 '../data/source/7_0.mkv',
 '../data/source/8_900.mkv',
 '../data/source/9_1800.mkv']

In [3]:
# splitting into file_time (in seconds) +- delta_seconds
file_time = 5.0 # seconds
delta = 0.5 # seconds
delta_step = 0.01 # seconds
min_sr = 8000
max_sr = 48000
resulting_durations = np.around(np.arange(file_time - delta, file_time + delta, delta_step), decimals = 2)
sampling_rates = np.arange(min_sr, max_sr, 1)

# duration in seconds, nned to pass file.mkv as last argument
check_file_duration_command = "ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1"

# now generate list of ffmpeg commands:
ffmpeg_commands = []
for file_index, file in enumerate(source_files):
    file_duration = int(float(subprocess.getoutput(f"{check_file_duration_command} {file}")))
    start_times = np.around(np.arange(0, file_duration, math.ceil(file_time + delta)), decimals = 1)
    start_times = start_times[1:-1] # let's remove first and last elements to be safe
    for start_index, start in enumerate(start_times):
        selected_duration = resulting_durations[randrange(len(resulting_durations))]
        selected_sample_rate = sampling_rates[randrange(len(sampling_rates))]
        output_file = f"../data/sliced/{file_index}_{int(start_index*math.ceil(file_time + delta))}_{selected_duration}_{selected_sample_rate}.wav"
        ffmpeg_commands += [f"\
        ffmpeg -y -i {file} \
            -ss {start} \
            -t {selected_duration} \
            -ab 320k \
            -ar {selected_sample_rate} \
            -ac 1 \
            -vn {output_file} \
            && chown 1000:1000 {output_file} \
            "]

In [4]:
" ".join(ffmpeg_commands[600].split())

'ffmpeg -y -i ../data/source/13_5400.mkv -ss 54 -t 4.63 -ab 320k -ar 42127 -ac 1 -vn ../data/sliced/4_48_4.63_42127.wav && chown 1000:1000 ../data/sliced/4_48_4.63_42127.wav'

In [5]:
%%time
### SET THREADS NUMBER 
nproc = 6
bash_command = "sleep 5" # for tests

semaphore = threading.Semaphore(nproc)
def run_command(cmd):
    with semaphore:
        os.system(cmd)
threads = []

for task in sorted(ffmpeg_commands):
    threads += [threading.Thread(target=run_command, args=(task,))]

for j in threads:
    j.start()
for j in threads:
    j.join()

CPU times: user 982 ms, sys: 51.9 s, total: 52.9 s
Wall time: 52min 34s
